In [30]:
 !pip install nltk

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [31]:
 !pip install -U sentence-transformers

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
from keras.models import Sequential
from keras.layers import Input, Embedding,TimeDistributed, LSTM, Dropout, Bidirectional, Dense 
import random
from nltk.corpus import wordnet 
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import plot_model, to_categorical
import nltk
nltk.download('punkt')  
from nltk.corpus import wordnet 
from tqdm import tqdm
import nltk
import subprocess
import logging

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet
nltk.download('punkt')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /kaggle/working/...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [33]:
file_path = "/kaggle/input/dataset65/src_65.txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    src = np.array([line.strip() for line in file])

file_path = "/kaggle/input/dataset65/top_decoupled_65.txt"

# Read data into an array of strings
with open(file_path, "r") as file:
    top_decoupled = np.array([line.strip() for line in file])

In [34]:
def preprosses(src,top_decoupled):
    word_tag=[]
    for src,trg in  zip(src,top_decoupled):
        out=extract_bio_tags(src,trg)
        word_tag.append(out)
    return word_tag

In [35]:
def extract_bio_tags(src,top_decoupled):
    # Parse the TOP-DECOUPLED structure
    matches = re.findall(r'\(NOT \([^()]+\) \)|\((?:[^()]+)\)|\(NOT \(COMPLEX_TOPPING \((?:.+?) \) \) \)', top_decoupled)
    ind=-1
    if random.random() < 0.5:
         ind=random.randint(0, len(src.split())-1)
    
    # Map words to BIO tags
    bio_tag_map = {}
    for i,match in enumerate(matches):
            
            match=re.sub(r'\(|\)','',match)
            match=match.strip()
            words= match.split()  # Split multi-word values
           
            if(words[0]=="NOT"):
               if("COMPLEX_TOPPING" in words):
                   words.remove("COMPLEX_TOPPING")
                   words.remove("NOT")
                   indx=words.index("TOPPING")
                   quantity=words[1:indx]
                   topping=words[indx+1:]
                   for i, word in enumerate(quantity):
                        if i == 0:
                            bio_tag_map[word]= "B-QUANTITY"  # Begin tag
                        else:
                            bio_tag_map[word]= "I-QUANTITY"  # Inside tag
                   for i, word in enumerate(topping):
                        if i == 0:
                            bio_tag_map[word]= "B-NOT-TOPPING"  # Begin tag
                        else:
                            bio_tag_map[word]= "I-NOT-TOPPING"  # Inside tag
                   continue
               else:
                   tag=words[0]+"-"+words[1]
                   words=words[2:]
            else:
                tag=words[0]
                words=words[1:]
         
            for i, word in enumerate(words):
                if i == 0:
                    bio_tag_map[word]= f"B-{tag}"  # Begin tag
                else:
                    bio_tag_map[word]= f"I-{tag}"  # Inside tag
       
                
            
  
    src_words = src.split()
    word_tag_pairs = [(word, bio_tag_map.get(word, "O")) for word in src_words]
    if ind!=-1:
        word_tag_pairs[ind]=("<UNK>",word_tag_pairs[ind][1])
    
    return word_tag_pairs






In [36]:
sentences =preprosses(src,top_decoupled)

In [37]:

maxlen = max([len(s) for s in sentences])
words = [w[0] for sentence in sentences for w in sentence]
words = list(set(words))
words.append("<pad>")
n_words = len(words)
print('Number of unique words:', n_words)


Number of unique words: 346


In [38]:


tags = [w[1] for sentence in sentences for w in sentence]
tags=list(set(tags))
print(tags)
n_tags = len(tags)
print('Number of unique Tags:', n_tags)

['I-NOT-STYLE', 'B-STYLE', 'I-NUMBER', 'I-CONTAINERTYPE', 'I-SIZE', 'B-TOPPING', 'I-NOT-TOPPING', 'B-DRINKTYPE', 'B-QUANTITY', 'I-QUANTITY', 'B-SIZE', 'B-NOT-TOPPING', 'B-NUMBER', 'I-VOLUME', 'B-NOT-STYLE', 'I-TOPPING', 'I-DRINKTYPE', 'B-VOLUME', 'I-STYLE', 'B-CONTAINERTYPE', 'O']
Number of unique Tags: 21


In [39]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
print(tag2idx)

{'I-NOT-STYLE': 0, 'B-STYLE': 1, 'I-NUMBER': 2, 'I-CONTAINERTYPE': 3, 'I-SIZE': 4, 'B-TOPPING': 5, 'I-NOT-TOPPING': 6, 'B-DRINKTYPE': 7, 'B-QUANTITY': 8, 'I-QUANTITY': 9, 'B-SIZE': 10, 'B-NOT-TOPPING': 11, 'B-NUMBER': 12, 'I-VOLUME': 13, 'B-NOT-STYLE': 14, 'I-TOPPING': 15, 'I-DRINKTYPE': 16, 'B-VOLUME': 17, 'I-STYLE': 18, 'B-CONTAINERTYPE': 19, 'O': 20}


In [40]:
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = sequence.pad_sequences(maxlen=maxlen+40, sequences=X, padding="post",value=n_words - 1)
y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = sequence.pad_sequences(maxlen=maxlen+40, sequences=y, padding="post", value=tag2idx['O'])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])
print('X shape', X.shape, 'y shape', y.shape)

X shape (15704, 62) y shape (15704, 62, 21)


In [41]:
class config():
    VOCAB = n_words
    MAX_LEN = maxlen+40
    N_OUPUT = n_tags
    
    
    EMBEDDING_VECTOR_LENGTH = 256
    N_LSTM_CELLS = 150
    RECURRENT_DROPOUT = 0.4
    
    OUTPUT_ACTIVATION = 'softmax'
    
    LOSS = 'categorical_crossentropy'
    OPTIMIZER = 'adam'
    METRICS = ['accuracy']
    
    MAX_EPOCHS = 5
    

In [42]:
X_one_hot = np.array([to_categorical(seq, num_classes=config.VOCAB) for seq in X])

In [43]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7)
filepath = "model.keras"
ckpt = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
rlp = ReduceLROnPlateau(monitor='loss', patience=3, verbose=1)

model = Sequential()
model.add(
    Bidirectional(
        LSTM(config.N_LSTM_CELLS, return_sequences=True, recurrent_dropout=config.RECURRENT_DROPOUT),
        input_shape=(config.MAX_LEN, config.VOCAB)
    )
)
model.add(
    TimeDistributed(
        Dense(config.N_OUPUT, activation=config.OUTPUT_ACTIVATION)
    )
)
model.compile(loss=config.LOSS, optimizer=config.OPTIMIZER, metrics=config.METRICS)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [44]:
history = model.fit(x=X_one_hot, y=y, validation_split=0.1,
                    callbacks=[es, ckpt, rlp], epochs=config.MAX_EPOCHS)

Epoch 1/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.8923 - loss: 0.5476
Epoch 1: loss improved from inf to 0.27528, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 61s 127ms/step - accuracy: 0.8924 - loss: 0.5469 - val_accuracy: 0.9891 - val_loss: 0.0472 - learning_rate: 0.0010
Epoch 2/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.9907 - loss: 0.0396
Epoch 2: loss improved from 0.27528 to 0.03111, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 55s 124ms/step - accuracy: 0.9907 - loss: 0.0396 - val_accuracy: 0.9957 - val_loss: 0.0180 - learning_rate: 0.0010
Epoch 3/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9957 - loss: 0.0171
Epoch 3: loss improved from 0.03111 to 0.01553, saving model to model.keras
442/442 ━━━━━━━━━━━━━━━━━━━━ 55s 124ms/step - accuracy: 0.9957 - loss: 0.0171 - val_accuracy: 0.9975 - val_loss: 0.0114 - learning_rate: 0.0010
Epoch 4/5
442/442 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9973 - loss: 0.0

In [49]:


def evaluate(test_src, test_top):
    true_len = 0
    true = 0

    for src, top in zip(test_src, test_top):
        src_words_original = src.split()
        if len(src_words_original) > config.MAX_LEN:
            continue
        
        true_len += 1

        # Replace unknown words with <UNK>
        for word in src_words_original:
            if word not in words:
                pattern = rf"(?<=\s){word}(?=\s)"
                src = re.sub(pattern, "<UNK>", src)

        src_words = src.split()
        src_indices = [word2idx.get(w, word2idx.get("<UNK>", -1)) for w in src_words]

        # Pad to max length
        pad_token = word2idx.get("<pad>")
        src_indices = src_indices + [pad_token] * (config.MAX_LEN - len(src_indices))

        # One-hot encode the indices
        src_one_hot = to_categorical(src_indices, num_classes=config.VOCAB)
        src_one_hot = np.expand_dims(src_one_hot, axis=0)  # Add batch dimension

        # Get predictions from the model
        predictions = model.predict(src_one_hot)
        predictions = np.argmax(predictions, axis=-1)  # Convert to tag indices

        idx2tag = {v: k for k, v in tag2idx.items()}
        predicted_tags = [idx2tag[i] for i in predictions[0]]

        # Prepare gold labels for comparison
        matches = re.findall(r'\(NOT \([^()]+\) \)|\((?:[^()]+)\)|\(NOT \(COMPLEX_TOPPING \((?:.+?) \) \) \)', top)
        bio_tag_map = {}
        for match in matches:
            match = re.sub(r'\(|\)', '', match).strip()
            uu = match.split()

            if uu[0] == "NOT":
                if "COMPLEX_TOPPING" in uu:
                    uu.remove("COMPLEX_TOPPING")
                    uu.remove("NOT")
                    indx = uu.index("TOPPING")
                    quantity = uu[1:indx]
                    topping = uu[indx + 1:]
                    for i, word in enumerate(quantity):
                        bio_tag_map[word] = "B-QUANTITY" if i == 0 else "I-QUANTITY"
                    for i, word in enumerate(topping):
                        bio_tag_map[word] = "B-NOT-TOPPING" if i == 0 else "I-NOT-TOPPING"
                    continue
                else:
                    tag = uu[0] + "-" + uu[1]
                    uu = uu[2:]
            else:
                tag = uu[0]
                uu = uu[1:]

            for i, word in enumerate(uu):
                bio_tag_map[word] = f"B-{tag}" if i == 0 else f"I-{tag}"

        word_tag_pairs = [(word, bio_tag_map.get(word, "O")) for word in src_words_original]
        word_tag_pairs = [pair[1] for pair in word_tag_pairs]
        word_tag_pairs = word_tag_pairs + ["O"] * (config.MAX_LEN - len(word_tag_pairs))

        acc = [a == b for a, b in zip(word_tag_pairs, predicted_tags)]
        if all(acc):
            true += 1

    print("Accuracy:", true / true_len * 100, "%")


In [50]:
dev_data = pd.read_json("/kaggle/input/pizza-dataset/PIZZA_dev.json", lines=True)
src_test=dev_data["dev.SRC"].to_numpy()
top_test=dev_data["dev.TOP"].to_numpy()


In [51]:
evaluate(src_test,top_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━

In [48]:

import json

with open("word2idx_model1.json", "w") as file:
    json.dump(word2idx,file)
with open("tag2idx_model1.json", "w") as file:
    json.dump(tag2idx,file)